# Lab 5

**Submission instructions**

Submit the Colab link to your notebook in Canvas. In addition, take screenshots of the map for each question and submit them to Canvas as well.

In [8]:
import ee
import geemap

In [9]:
geemap.ee_initialize()

## Question 1

Visualize the [USGS Watershed Boundary Dataset](https://developers.google.com/earth-engine/datasets/catalog/USGS_WBD_2017_HUC04) with outline color only, no fill color.

In [10]:
watersheds = ee.FeatureCollection("USGS/WBD/2017/HUC04")

In [11]:
usa_params = {'geodesic': False,
 'type': 'Polygon',
 'coordinates': [[[-131.382047, 21.187133],
   [-131.382047, 53.054525],
   [-61.407681, 53.054525],
   [-61.407681, 21.187133],
   [-131.382047, 21.187133]]]}

usa = ee.Feature(usa_params)

In [12]:
style = {
    'color': '0000ffff',
    'width': 1,
    'lineType': 'solid',
    'fillColor': '00000000',
}

In [13]:
m = geemap.Map()
m.addLayer(watersheds.style(**style))
m.centerObject(usa)
text = "Made by Ilse Paniagua"
m.add_text(text)
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

![](https://i.imgur.com/PLlNFq3.png)

## Question 2 

Filter the USGS Watershed Boundary dataset and select the watershed that intersects the county of your choice.

In [14]:
# Add county dataset
tiger = ee.FeatureCollection("TIGER/2018/Counties")
washtenaw_filter =[ee.Filter.eq('STATEFP', '26'), ee.Filter.eq('NAME', 'Washtenaw')]
washtenaw_county = tiger.filter(washtenaw_filter)

In [15]:
filtred_watersheds = watersheds.filterBounds(washtenaw_county)

In [16]:
p = geemap.Map()
p.centerObject(filtred_watersheds)
p.addLayer(filtred_watersheds)
p.add_text(text)
p

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

![](https://i.imgur.com/F2QfqZu.png)

## Question 3

Clip the [USGS 3DEP 10m DEM](https://developers.google.com/earth-engine/datasets/catalog/USGS_3DEP_10m) with the watershed that intersects the county of your choice. Display the DEM with a proper color palette and color bar.

In [17]:
dem = ee.Image("USGS/3DEP/10m")
filtered_dem = dem.clip(washtenaw_county)

In [18]:
dem_viz = {
    "palette": "terrain",
    "min": 0,
    "max": 500,
    "opacity": 0.7
}

In [19]:
legends = geemap.builtin_legends
for legend in legends:
    print(legend)

NLCD
ESA_WorldCover
ESRI_LandCover
ESRI_LandCover_TS
Dynamic_World
NWI
MODIS/051/MCD12Q1
MODIS/006/MCD12Q1
GLOBCOVER
JAXA/PALSAR
Oxford
AAFC/ACI
COPERNICUS/CORINE/V20/100m
COPERNICUS/Landcover/100m/Proba-V/Global
USDA/NASS/CDL
ALOS_landforms


In [20]:
style = {
    'color': 'ff0000ff',
    'width': 2,
    'lineType': 'solid',
    'fillColor': '00000000',
}

In [21]:
q = geemap.Map()
q.addLayer(filtered_dem, dem_viz, "DEM")
q.addLayer(washtenaw_county.style(**style))
q.centerObject(washtenaw_county)
q.add_colorbar(dem_viz, label="Elevation (m)", orientation="horizontal")
q.add_text(text)
q

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

![](https://i.imgur.com/okR39pf.png)

## Question 4

Use the [USGS National Land Cover Database](https://developers.google.com/earth-engine/datasets/catalog/USGS_NLCD_RELEASES_2019_REL_NLCD) and [US Census States](https://developers.google.com/earth-engine/datasets/catalog/TIGER_2018_States) to create a split-panel map for visualizing land cover change (2001-2019) for a US state of your choice. Make sure you add the NLCD legend to the map.

In [22]:
states = ee.FeatureCollection("TIGER/2018/States")
mi = states.filter(ee.Filter.eq("NAME", "Michigan"))
mi

In [23]:
nlcd_collection = ee.ImageCollection("USGS/NLCD_RELEASES/2019_REL/NLCD").select("landcover")
nlcd_2001 = nlcd_collection.filterBounds(mi).filterDate("2001-01-01", "2001-12-31").first()
nlcd_2019 = nlcd_collection.filterBounds(mi).filterDate("2019-01-01", "2019-12-31").first()

In [24]:
nlcd_2001

In [25]:
nlcd_viz = {
"bands": ["landcover"]
 }

In [26]:
left_layer = geemap.ee_tile_layer(nlcd_2001, nlcd_viz, "NLCD 2001")
right_layer = geemap.ee_tile_layer(nlcd_2019, nlcd_viz, "NLCD 2019")

s = geemap.Map()

s.centerObject(mi)
s.split_map(left_layer, right_layer, left_label="2001", right_label="2019")
s.add_text(text)
s.add_legend(builtin_legend="NLCD", )
s

Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…

![](https://i.imgur.com/Au7Q5Ln.png)

## Questions 5

Download OpenStreetMap data for a city of your choice and visualize the city boundary and restaurants in the city.

In [27]:
# aa = m.user_roi
# aa.getInfo()

aa = {'geodesic': False,
 'type': 'Polygon',
 'coordinates': [[[-83.807607, 42.217171],
   [-83.807607, 42.328949],
   [-83.640721, 42.328949],
   [-83.640721, 42.217171],
   [-83.807607, 42.217171]]]}

In [29]:
m = geemap.Map(toolbar_control=False, layers_control=True)
m.add_osm_from_geocode("Ann Arbor", layer_name="AA")
m.add_osm_from_address(
    address="Ann Arbor", tags={"amenity": "restaurant"}, dist=1500, layer_name="Ann Arbor restaurants"
)
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

![](https://i.imgur.com/AUlO1CV.png)